In [8]:
import zipfile

with zipfile.ZipFile('../Test.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [21]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from skimage.feature import hog
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [12]:
suffix = '.pgm'
kaggle_files = os.listdir('Test')
pgm_kaggle_files = [filename for filename in kaggle_files if filename.endswith(suffix)]

In [ ]:
len(pgm_kaggle_files)

8000

In [18]:

def imgs_to_hog(imgs):
    hog_features_test = []
    for img in tqdm(imgs, desc='Extrayendo HOG features'):
        fd = hog(img, visualize=False)
        hog_features_test.append(fd)

        #transformamos en array y stackeamos
    ret_array = np.array(hog_features_test)
    ret_array = np.vstack(hog_features_test)

    return ret_array

In [16]:
kaggle_patches = []
for filename in tqdm(pgm_kaggle_files):
  path = 'Test/' + filename
  with open(path, 'rb') as pgmf:
      image = plt.imread(pgmf)
  kaggle_patches.append(image)

100%|██████████| 8000/8000 [02:07<00:00, 62.51it/s]


In [27]:
pgm_kaggle_files_id = []

for filename in tqdm(pgm_kaggle_files):
  pgm_kaggle_files_id.append(filename[5:-4])

100%|██████████| 8000/8000 [00:00<00:00, 2355358.14it/s]


In [22]:
# Aquí debe utilizar su modelo
model = joblib.load('../modelos/xgb_model.pkl')

In [23]:
# Aquí debe procesar las imagenes de pgm_kaggle_files
pca_model = joblib.load('../modelos/pca_final_2025-06-08.pkl')
pgm_kaggle_array = np.stack(kaggle_patches)
pgm_kaggle_array = pgm_kaggle_array / 255.0  # Normalizar las imágenes
pgm_kaggle_array = imgs_to_hog(pgm_kaggle_array)  # Extraer características HOG
kaggle_images_flatten = np.array([im.flatten() for im in tqdm(pgm_kaggle_array, desc='Construyendo X')])
X_pca_kag = pca_model.transform(kaggle_images_flatten)


Construyendo X: 100%|██████████| 8000/8000 [00:00<00:00, 74177.21it/s]


In [24]:
X_pca_kag.shape

(8000, 450)

In [25]:
# Predicción
y_kag = model.predict(X_pca_kag)

In [28]:
# Predicción + ID
y_kag_dic = {pgm_kaggle_files_id[i]: y_kag[i] for i in range(len(pgm_kaggle_files_id))}
kaggle_hat = pd.DataFrame(list(y_kag_dic.items()), columns=['id', 'target_feature'])
kaggle_hat['id'] = kaggle_hat['id'].astype(int)
kaggle_hat['target_feature'] = kaggle_hat['target_feature'].astype(int)
kaggle_hat.sort_values(by='id', inplace=True)

In [29]:
kaggle_hat.head()

,id,target_feature
0,0,0
1,1,0
1112,2,0
2223,3,0
3334,4,0


In [30]:
kaggle_hat.value_counts('target_feature')

target_feature
0    7595
1     405
Name: count, dtype: int64

In [31]:
# Guardamos en un .csv para subir a kaggle
submission_name = 'prueba_2025-06-08'
kaggle_hat.to_csv('../submissions/'+ submission_name + '.csv', index=False)